## Utforsk K-Means-klustering med R og prinsipper for ryddige data.

### [**Quiz før forelesning**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

I denne leksjonen vil du lære hvordan du oppretter klynger ved hjelp av Tidymodels-pakken og andre pakker i R-økosystemet (vi kaller dem venner 🧑‍🤝‍🧑), samt det nigerianske musikkdatasettet du importerte tidligere. Vi skal dekke det grunnleggende om K-Means for klustering. Husk at, som du lærte i den forrige leksjonen, finnes det mange måter å jobbe med klynger på, og metoden du bruker avhenger av dataene dine. Vi skal prøve K-Means, da det er den mest vanlige klusteringsteknikken. La oss komme i gang!

Begreper du vil lære om:

-   Silhuettscore

-   Albuemetoden

-   Inerti

-   Varians

### **Introduksjon**

[K-Means-klustering](https://wikipedia.org/wiki/K-means_clustering) er en metode som stammer fra signalbehandling. Den brukes til å dele og gruppere data i `k klynger` basert på likheter i deres egenskaper.

Klyngene kan visualiseres som [Voronoi-diagrammer](https://wikipedia.org/wiki/Voronoi_diagram), som inkluderer et punkt (eller 'frø') og dets tilsvarende område.

<p >
   <img src="../../images/voronoi.png"
   width="500"/>
   <figcaption>Infografikk av Jen Looper</figcaption>


K-Means-klustering har følgende steg:

1.  Dataanalytikeren starter med å spesifisere ønsket antall klynger som skal opprettes.

2.  Deretter velger algoritmen tilfeldig K observasjoner fra datasettet som skal fungere som de første sentrene for klyngene (dvs. sentroidene).

3.  Deretter blir hver av de gjenværende observasjonene tilordnet sin nærmeste sentroid.

4.  Deretter beregnes det nye gjennomsnittet for hver klynge, og sentroiden flyttes til gjennomsnittet.

5.  Nå som sentrene er blitt oppdatert, sjekkes hver observasjon igjen for å se om den kan være nærmere en annen klynge. Alle objektene blir tilordnet på nytt ved hjelp av de oppdaterte klynge-gjennomsnittene. Stegene for klyngetildeling og oppdatering av sentroidene gjentas iterativt til klyngetildelingene slutter å endre seg (dvs. når konvergens er oppnådd). Vanligvis avsluttes algoritmen når hver ny iterasjon resulterer i ubetydelig bevegelse av sentroidene, og klyngene blir statiske.

<div>

> Merk at på grunn av tilfeldigheten i valg av de første k observasjonene som brukes som start-sentroidene, kan vi få litt forskjellige resultater hver gang vi bruker prosedyren. Av denne grunn bruker de fleste algoritmer flere *tilfeldige starter* og velger iterasjonen med lavest WCSS. Det anbefales derfor sterkt å alltid kjøre K-Means med flere verdier av *nstart* for å unngå et *uønsket lokalt optimum.*

</div>

Denne korte animasjonen ved hjelp av [illustrasjoner](https://github.com/allisonhorst/stats-illustrations) av Allison Horst forklarer klusteringsprosessen:

<p >
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>Illustrasjon av @allison_horst</figcaption>



Et grunnleggende spørsmål som oppstår i klustering er dette: hvordan vet du hvor mange klynger du skal dele dataene dine inn i? En ulempe med å bruke K-Means er at du må fastsette `k`, altså antallet `sentroider`. Heldigvis hjelper `albuemetoden` med å estimere en god startverdi for `k`. Du skal prøve det om et øyeblikk.

### 

**Forutsetning**

Vi fortsetter der vi avsluttet i [forrige leksjon](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb), hvor vi analyserte datasettet, laget mange visualiseringer og filtrerte datasettet til interessante observasjoner. Sørg for å sjekke det ut!

Vi trenger noen pakker for å fullføre denne modulen. Du kan installere dem slik: `install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

Alternativt sjekker skriptet nedenfor om du har de nødvendige pakkene for å fullføre denne modulen og installerer dem for deg hvis noen mangler.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


La oss komme i gang!

## 1. En dans med data: Begrens deg til de 3 mest populære musikksjangrene

Dette er en oppsummering av hva vi gjorde i forrige leksjon. La oss analysere og bearbeide litt data!


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 Det gikk bra!

## 2. Mer utforsking av data.

Hvor ren er denne dataen? La oss sjekke for uteliggere ved hjelp av boksplott. Vi vil fokusere på numeriske kolonner med færre uteliggere (selv om du kan fjerne uteliggere). Boksplott kan vise rekkevidden av dataen og vil hjelpe med å velge hvilke kolonner som skal brukes. Merk at boksplott ikke viser varians, et viktig element for god klustringsvennlig data. Vennligst se [denne diskusjonen](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot) for videre lesing.

[Boksplott](https://en.wikipedia.org/wiki/Box_plot) brukes til å grafisk fremstille fordelingen av `numerisk` data, så la oss starte med å *velge* alle numeriske kolonner sammen med de populære musikksjangrene.


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


Se hvordan utvalgsfunksjonen `where` gjør dette enkelt 💁? Utforsk andre slike funksjoner [her](https://tidyselect.r-lib.org/).

Siden vi skal lage et boksplott for hver numeriske egenskap og ønsker å unngå å bruke løkker, la oss omformatere dataene våre til et *lengre* format som lar oss dra nytte av `facets` - delplott som hver viser en delmengde av dataene.


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


Mye lengre! Nå er det tid for noen `ggplots`! Så hvilken `geom` skal vi bruke?


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


Enkelt-gg!

Nå kan vi se at disse dataene er litt støyete: ved å observere hver kolonne som et boksplott, kan du se uteliggere. Du kunne gått gjennom datasettet og fjernet disse uteliggerne, men det ville gjort dataene ganske minimale.

For nå, la oss velge hvilke kolonner vi skal bruke til vår klyngingsøvelse. La oss velge de numeriske kolonnene med lignende intervaller. Vi kunne kodet `artist_top_genre` som numerisk, men vi dropper den foreløpig.


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## 3. Beregning av k-means klynging i R

Vi kan beregne k-means i R ved hjelp av den innebygde `kmeans`-funksjonen, se `help("kmeans()")`. `kmeans()`-funksjonen aksepterer en data frame med kun numeriske kolonner som sitt primære argument.

Det første steget når man bruker k-means klynging er å spesifisere antall klynger (k) som skal genereres i den endelige løsningen. Vi vet at det er 3 musikksjangre som vi har skilt ut fra datasettet, så la oss prøve med 3:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


Kmeans-objektet inneholder flere biter med informasjon som er godt forklart i `help("kmeans()")`. For nå, la oss fokusere på noen få. Vi ser at dataene har blitt gruppert i 3 klynger med størrelser 65, 110, 111. Utdataene inneholder også klyngesentrene (gjennomsnitt) for de 3 gruppene på tvers av de 5 variablene.

Klyngingsvektoren er klyngeoppgaven for hver observasjon. La oss bruke funksjonen `augment` for å legge til klyngeoppgaven i det opprinnelige datasettet.


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


Perfekt, vi har nettopp delt datasettet vårt inn i 3 grupper. Så, hvor bra er vår klyngedeling 🤷? La oss ta en titt på `Silhouette score`.

### **Silhouette score**

[Silhouette-analyse](https://en.wikipedia.org/wiki/Silhouette_(clustering)) kan brukes til å studere avstanden mellom de resulterende klyngene. Denne scoren varierer fra -1 til 1, og hvis scoren er nær 1, er klyngen tett og godt adskilt fra andre klynger. En verdi nær 0 representerer overlappende klynger med prøver som ligger veldig nær beslutningsgrensen til naboklyngene. [kilde](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam).

Metoden for gjennomsnittlig silhouette beregner gjennomsnittlig silhouette for observasjoner for ulike verdier av *k*. En høy gjennomsnittlig silhouette-score indikerer en god klyngedeling.

`silhouette`-funksjonen i cluster-pakken brukes til å beregne gjennomsnittlig silhouette-bredde.

> Silhouetten kan beregnes med hvilken som helst [avstand](https://en.wikipedia.org/wiki/Distance "Distance")-metrik, som [Euklidsk avstand](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance") eller [Manhattan-avstand](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance") som vi diskuterte i [forrige leksjon](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb).


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


Vår score er **.549**, altså midt på treet. Dette indikerer at dataene våre ikke er spesielt godt egnet for denne typen klyngedannelse. La oss se om vi kan bekrefte denne antagelsen visuelt. [factoextra-pakken](https://rpkgs.datanovia.com/factoextra/index.html) tilbyr funksjoner (`fviz_cluster()`) for å visualisere klyngedannelse.


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


Overlappen i klyngene indikerer at dataene våre ikke er spesielt godt egnet for denne typen klynging, men la oss fortsette.

## 4. Bestemme optimale klynger

Et grunnleggende spørsmål som ofte oppstår i K-Means klynging er dette - uten kjente klasseetiketter, hvordan vet du hvor mange klynger du skal dele dataene dine inn i?

En måte vi kan prøve å finne ut av dette på er å bruke et datasett til å `lage en serie med klyngemodeller` med et økende antall klynger (f.eks. fra 1-10), og evaluere klyngemetrikker som **Silhouette score.**

La oss bestemme det optimale antallet klynger ved å beregne klyngealgoritmen for forskjellige verdier av *k* og evaluere **Within Cluster Sum of Squares** (WCSS). Den totale within-cluster sum of squares (WCSS) måler hvor kompakt klyngene er, og vi ønsker at den skal være så liten som mulig, der lavere verdier betyr at datapunktene er nærmere hverandre.

La oss undersøke effekten av forskjellige valg av `k`, fra 1 til 10, på denne klyngingen.


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


Nå som vi har den totale sum av kvadrater innenfor klyngene (tot.withinss) for hver klyngealgoritme med sentrum *k*, bruker vi [albuemetoden](https://en.wikipedia.org/wiki/Elbow_method_(clustering)) for å finne det optimale antallet klynger. Metoden består i å plotte WCSS som en funksjon av antallet klynger, og velge [albuen på kurven](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Albuen på kurven") som antallet klynger som skal brukes.


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


Plottet viser en stor reduksjon i WCSS (altså større *tetthet*) når antall klynger øker fra én til to, og en ytterligere merkbar reduksjon fra to til tre klynger. Etter dette blir reduksjonen mindre tydelig, noe som resulterer i en `knekk` 💪 i diagrammet rundt tre klynger. Dette er en god indikasjon på at det finnes to til tre rimelig godt separerte klynger av datapunkter.

Vi kan nå gå videre og hente ut klyngemodellen der `k = 3`:

> `pull()`: brukes til å hente ut en enkelt kolonne
>
> `pluck()`: brukes til å indeksere datastrukturer som lister


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


Flott! La oss visualisere klyngene vi har fått. Hva med litt interaktivitet ved hjelp av `plotly`?


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


Kanskje vi ville ha forventet at hver klynge (representert med forskjellige farger) hadde tydelige sjangre (representert med forskjellige former).

La oss se på modellens nøyaktighet.


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


Denne modellens nøyaktighet er ikke dårlig, men heller ikke fantastisk. Det kan være at dataene ikke egner seg godt for K-Means Clustering. Disse dataene er for ubalanserte, har for lite korrelasjon, og det er for stor variasjon mellom verdiene i kolonnene til å danne gode klynger. Faktisk er det sannsynlig at klyngene som dannes er sterkt påvirket eller skjevfordelt av de tre sjangerkategoriene vi definerte ovenfor.

Likevel, dette var en ganske lærerik prosess!

I Scikit-learn-dokumentasjonen kan du se at en modell som denne, med klynger som ikke er veldig godt avgrenset, har et 'varians'-problem:

<p >
   <img src="../../images/problems.png"
   width="500"/>
   <figcaption>Infografikk fra Scikit-learn</figcaption>



## **Varians**

Varians er definert som "gjennomsnittet av de kvadrerte avvikene fra gjennomsnittet" [kilde](https://www.mathsisfun.com/data/standard-deviation.html). I konteksten av dette klyngeproblemet refererer det til data der verdiene i datasettet vårt har en tendens til å avvike litt for mye fra gjennomsnittet.

✅ Dette er et flott øyeblikk for å tenke på alle måtene du kan løse dette problemet. Justere dataene litt mer? Bruke andre kolonner? Bruke en annen algoritme? Hint: Prøv [å skalere dataene dine](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/) for å normalisere dem og teste andre kolonner.

> Prøv denne '[varianskalkulatoren](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)' for å forstå konseptet litt bedre.

------------------------------------------------------------------------

## **🚀Utfordring**

Bruk litt tid på denne notatboken og juster parametere. Kan du forbedre modellens nøyaktighet ved å rense dataene mer (for eksempel fjerne uteliggere)? Du kan bruke vekter for å gi mer vekt til visse datasettprøver. Hva annet kan du gjøre for å lage bedre klynger?

Hint: Prøv å skalere dataene dine. Det finnes kommentert kode i notatboken som legger til standard skalering for å få datakolonnene til å ligne hverandre mer når det gjelder rekkevidde. Du vil oppdage at selv om silhuettscoren går ned, jevner 'knekkpunktet' i albuegrafen seg ut. Dette skyldes at når dataene ikke er skalert, får data med mindre varians mer vekt. Les litt mer om dette problemet [her](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226).

## [**Quiz etter forelesning**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **Gjennomgang & Selvstudium**

-   Ta en titt på en K-Means Simulator [som denne](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/). Du kan bruke dette verktøyet til å visualisere prøvedatapunkter og bestemme deres sentroider. Du kan redigere dataenes tilfeldighet, antall klynger og antall sentroider. Hjelper dette deg med å få en idé om hvordan dataene kan grupperes?

-   Ta også en titt på [dette handoutet om K-Means](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) fra Stanford.

Vil du prøve ut dine nyervervede klyngeferdigheter på datasett som egner seg godt for K-Means clustering? Se:

-   [Tren og evaluer klyngemodeller](https://rpubs.com/eR_ic/clustering) ved hjelp av Tidymodels og venner

-   [K-means Cluster Analysis](https://uc-r.github.io/kmeans_clustering), UC Business Analytics R Programming Guide

- [K-means clustering med prinsipper for ryddige data](https://www.tidymodels.org/learn/statistics/k-means/)

## **Oppgave**

[Prøv forskjellige klynge-metoder](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## TAKK TIL:

[Jen Looper](https://www.twitter.com/jenlooper) for å ha laget den originale Python-versjonen av dette modulen ♥️

[`Allison Horst`](https://twitter.com/allison_horst/) for å ha laget de fantastiske illustrasjonene som gjør R mer innbydende og engasjerende. Finn flere illustrasjoner i hennes [galleri](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

Lykke til med læringen,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="500"/>
   <figcaption>Kunstverk av @allison_horst</figcaption>



---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi tilstreber nøyaktighet, vennligst vær oppmerksom på at automatiske oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for eventuelle misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
